In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
from matplotlib.patches import Polygon
import matplotlib
from matplotlib.collections import PatchCollection
import mpl_toolkits
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
from shapely.prepared import prep
from descartes import PolygonPatch
import matplotlib.pyplot as plt
from collections import defaultdict

% matplotlib inline

In [ ]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 200

In [ ]:
df=pd.read_csv('data/data_seattle.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
#df['proptype']=df['prop_type']

In [ ]:
#df[df['mls_prop_type']=='32 - Townhouse']=
#                                df[df['mls_prop_type']=='32 - Townhouse'].replace({'proptypee':{'Residential':'Townhouse'}})

In [ ]:
#df.to_csv('data/data_seattle.csv')

In [ ]:
def plot_prepper(m, shapefilename, df_key):
    '''
    Generate dataframe for plotting
    '''
    _out = m.readshapefile(shapefilename, name='seattle', drawbounds=False, color='none', zorder=2)
    # set up a map dataframe for neighborhood outlines
    cent_lons = [float((Polygon(points).centroid.wkt).split()[1][1:]) for points in m.seattle]
    cent_lats = [float((Polygon(points).centroid.wkt).strip(')').split()[2]) for points in m.seattle]

    df_poly = pd.DataFrame({
        'poly': [Polygon(points) for points in m.seattle],
        'name': [item[df_key] for item in m.seattle_info],
    })
    df_poly['centroid_1'] = cent_lons
    df_poly['centroid_2'] = cent_lats
    return _out, df_poly

In [ ]:
# Read in neighborhood shape file
hood_shapefilename = 'neighborhoods/WGS84/Neighborhoods'
hood_coords = (-122.43595589761988, 47.49551362397606, 
               -122.23604445458412, 47.73416494684497)

w, h = hood_coords[2] - hood_coords[0], hood_coords[3] - hood_coords[1]
extra = 0.005

In [ ]:
# Initialize the plot
figwidth = 12
fig = plt.figure(figsize=(figwidth, figwidth*h/w))
ax = fig.add_subplot(111, axisbg='w', frame_on=False)
m = Basemap(projection='tmerc', ellps='WGS84',
            lon_0=np.mean([hood_coords[0], hood_coords[2]]),
            lat_0=np.mean([hood_coords[1], hood_coords[3]]),
            llcrnrlon=hood_coords[0] - extra * w,
            llcrnrlat=hood_coords[1] - (extra * h),
            urcrnrlon=hood_coords[2] + extra * w,
            urcrnrlat=hood_coords[3] + (extra * h),
            resolution='i',  suppress_ticks=True)
# Get dataframe for plotting
_out, df_map = plot_prepper(m, hood_shapefilename, 'S_HOOD')
nbr_names = df_map['name'].unique()

# Draw neighborhoods with grey outlines
df_map['patches'] = df_map['poly'].map(lambda x: PolygonPatch(x, ec='#050505', lw=.8, alpha=1., zorder=4))
pc = PatchCollection(df_map['patches'], match_original=True)
pc.set_facecolor('w')
ax.add_collection(pc)

# Scale the map
_=m.drawmapscale(hood_coords[0] + 0.08, hood_coords[1] + -0.01, 
               hood_coords[0], hood_coords[1], 10., barstyle='fancy',
               fillcolor1='w', fillcolor2='#555555', zorder=5, ax=ax,)

lons = df.longitude
lats = df.latitude

pt = []
xs = []
ys = []

'''
This plots all the latitude/longitude points
'''
for mapped_x, mapped_y in zip(lons, lats):
    xpt, ypt = m(mapped_x, mapped_y)
    plt.plot(xpt, ypt, 'v', markersize=5, alpha=0.3, color='#483070')
    pt.append(Point(xpt, ypt))
    xs.append(xpt)
    ys.append(ypt)

'''
Checks the neighborhood of point
'''
nmlist = []
# Creates list of neighborhoods
for polygon in df_map['poly']:
    nm = df_map.loc[df_map['poly'] == polygon, 'name'].iloc[0]
    if len(nm.strip()) < 4 or nm in nmlist:
        continue
    nmlist.append(nm)
    
# Add neighborhood names
nml = []
for i, lon in enumerate(df_map['centroid_1'].values):
    if df_map['name'].values[i] in nml:
        continue
    nml.append(df_map['name'].values[i])
    if len(df_map['name'].values[i]) > 3:
        plt.text(lon, df_map['centroid_2'].values[i], df_map['name'].values[i], size=7, ha='center')

ax.axes.get_xaxis().set_visible(False)

In [ ]:
df.head(2)

In [ ]:
import pandas as pd
import numpy as np
df_reviews=pd.read_csv('data/street_reviews.csv')
df_reviews.drop('Unnamed: 0', axis=1, inplace=True)
df_sample=pd.read_csv('data/data_seattle.csv')
df_sample.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
minbed=2
maxbed=3
minbath=1
maxbath=2
proptype='Townhouse'
neighborhood='Atlantic'

In [ ]:
import system as S
recsys =S.ReccomnderSystem(df_sample,df_reviews)

In [ ]:
df_selected = df[df_sample['id'].isin(
    recsys.home_reccomender_dict(minbed, maxbed, minbath, maxbath, proptype, neighborhood))]

lats_selected=df_selected['latitude']
lons_selected=df_selected['longitude']
avg_lat=np.mean(lats_selected.values)
avg_lon=np.mean(lons_selected.values)

In [ ]:
#483070
#https://pypi.python.org/pypi/gmplot/1.0.5
import gmplot

#gmap = gmplot.GoogleMapPlotter(47.6062, -122.3321, 12)
gmap = gmplot.GoogleMapPlotter(avg_lat, avg_lon, 14)

#gmap.plot(lats.values, lons.values, 'cornflowerblue', edge_width=0.1)
#gmap.scatter(lats.values, lons.values, '#483070', size=40, marker=False)
gmap.scatter(lats_selected, lons_selected, 'g', size=0.00001, marker=True)
#gmap.heatmap(heat_lats, heat_lngs)

gmap.draw("mymap.html")